In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
WITH usa_customers AS
    (SELECT *
       FROM customer
      WHERE country = "USA")

SELECT *
  FROM usa_customers
 LIMIT 3;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3


In [4]:
%%sql
WITH tracks_and_genre AS
(
SELECT il.invoice_id,
       g.name genre,
       SUM(il.quantity) number_of_tracks_sold,
       SUM(CAST(il.quantity AS float)) / 
            (SELECT SUM(quantity)
               FROM invoice_line) percentage_tracks_sold
  FROM invoice_line il
  LEFT JOIN track t
    ON il.track_id = t.track_id
  LEFT JOIN genre g
    ON t.genre_id = g.genre_id
 GROUP BY genre )

SELECT *
  FROM tracks_and_genre;

Done.


invoice_id,genre,number_of_tracks_sold,percentage_tracks_sold
600,Alternative,117,0.024595333193188986
609,Alternative & Punk,492,0.10342652932520496
595,Blues,124,0.026066848854319948
586,Classical,47,0.009880176581879336
29,Drama,1,0.00021021652301870928
610,Easy Listening,74,0.015556022703384485
566,Electronica/Dance,55,0.01156190876602901
558,Heavy Metal,8,0.0016817321841496743
577,Hip Hop/Rap,33,0.006937145259617406
613,Jazz,121,0.025436199285263823


In [5]:
%%sql

SELECT 
       g.name genre,
       SUM(il.quantity) number_of_tracks_sold,
       SUM(CAST(il.quantity AS float)) / 
            (SELECT SUM(il.quantity)
               FROM invoice_line il
               LEFT JOIN invoice i
                 ON il.invoice_id = i.invoice_id
               LEFT JOIN customer c
                 ON i.customer_id = c.customer_id
              WHERE c.country = "USA") percentage_tracks_sold
  FROM invoice_line il
  LEFT JOIN track t
    ON il.track_id = t.track_id
  LEFT JOIN genre g
    ON t.genre_id = g.genre_id
  LEFT JOIN invoice i
    ON il.invoice_id = i.invoice_id
  LEFT JOIN customer c
    ON i.customer_id = c.customer_id
 WHERE c.country = "USA"
 GROUP BY genre
 ORDER BY number_of_tracks_sold DESC
 LIMIT 10;

Done.


genre,number_of_tracks_sold,percentage_tracks_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


## Top Selling Genres in the USA

Rock is by far the best selling genre in the UK, with over 50% of all tracks sold.  The next top sellers are Alternative and Punk and Metal.  The top 3 best selling albums are also from Rock genre.

In [6]:
%%sql
SELECT at.name artist_name,
       al.title album,
       g.name genre,
       SUM(il.quantity) album_sales
  FROM track t
  LEFT JOIN album al
    ON t.album_id = al.album_id
  LEFT JOIN artist at
    ON al.artist_id = at.artist_id
  LEFT JOIN genre g
    ON t.genre_id = g.genre_id
  LEFT JOIN invoice_line il
    ON t.track_id = il.track_id
  LEFT JOIN invoice i
    ON i.invoice_id = il.invoice_id
  LEFT JOIN customer c
    ON i.customer_id = c.customer_id
 WHERE c.country = "USA"
 GROUP BY album
 ORDER BY album_sales DESC
 LIMIT 3;


Done.


artist_name,album,genre,album_sales
Jimi Hendrix,Are You Experienced?,Rock,28
Nirvana,From The Muddy Banks Of The Wishkah [live],Rock,28
The Doors,The Doors,Rock,27


In [7]:
%%sql
SELECT e.first_name || " " || e.last_name employee_name,
       e.hire_date,
       ROUND(SUM(i.total), 2) total_sales,
       ROUND(AVG(i.total), 2) average_sale
  FROM employee e
 INNER JOIN customer c
    ON e.employee_id = c.support_rep_id
 INNER JOIN invoice i
    ON c.customer_id = i.customer_id
 WHERE title = "Sales Support Agent"
 GROUP BY employee_name

Done.


employee_name,hire_date,total_sales,average_sale
Jane Peacock,2017-04-01 00:00:00,1731.51,8.17
Margaret Park,2017-05-03 00:00:00,1584.0,7.4
Steve Johnson,2017-10-17 00:00:00,1393.92,7.41


## Sales made by Sales Support Agents

There are three sales support agents, Jane Peacock makes the most sales at \$1731.51.  She also makes the largest sale at \$8.17.  Margaret Park makes \$1584 in total sales with average sale of \$7.4.  Steve Johnson makes a similar average sale to Margaret at \$7.41, but because Steve has been working 5 months less than Margaret his total sales are only \$1393.92.  Jane has been working the longest, which may account somewhat for her higher sales.

In [8]:
%%sql

WITH countries_other AS
    (SELECT CASE
               WHEN (
                      SELECT COUNT(*)
                        FROM customer 
                       WHERE country = c.country) = 1
                      THEN "Other"
                      ELSE c.country
                      END country,
               c.customer_id,
               i.*
      FROM customer c
      LEFT JOIN invoice i
        ON c.customer_id = i.customer_id)

SELECT country,
       COUNT(DISTINCT(customer_id)) number_of_customers,
       ROUND(SUM(total), 2) total_sales,
       ROUND(AVG(total), 2) avg_sale,
       ROUND(SUM(total) / COUNT(DISTINCT(customer_id))) avg_customer_total,
       CASE
          WHEN country = "Other" THEN 1
          ELSE 0
          END AS sort
  FROM countries_other
 GROUP BY country
 ORDER BY sort, number_of_customers DESC

Done.


country,number_of_customers,total_sales,avg_sale,avg_customer_total,sort
USA,13,1040.49,7.94,80.0,0
Canada,8,535.59,7.05,67.0,0
Brazil,5,427.68,7.01,86.0,0
France,5,389.07,7.78,78.0,0
Germany,4,334.62,8.16,84.0,0
United Kingdom,3,245.52,8.77,82.0,0
Czech Republic,2,273.24,9.11,137.0,0
India,2,183.15,8.72,92.0,0
Portugal,2,185.13,6.38,93.0,0
Other,15,1094.94,7.45,73.0,1


## Country Customers Sales

The country with the most customers is the USA with almost twice as many as Canada, this is also reflected in total sales.  Some other countries have higher average sales or average customer total sales, like Czech Republic.  Countries with only 1 customer have all been placed in "Other" category, and together they have sales of 1094, slightly higher than the USA, but this is the combined total from 15 countries.

In [28]:
%%sql

WITH invoices_albums AS
    (SELECT a.album_id, 
            t.track_id album_track,
            il.invoice_id,
            il.track_id invoice_track
       FROM album a
       LEFT JOIN track t
         ON a.album_id = t.album_id
       LEFT JOIN invoice_line il
         ON t.track_id = il.track_id)
    
SELECT 

Done.


album_id,album_track,invoice_id,invoice_track,album_in_invoice
91,1158,1,1158,True
91,1159,1,1159,True
91,1160,1,1160,True
91,1161,1,1161,True
91,1162,1,1162,True
91,1163,1,1163,True
91,1164,1,1164,True
91,1165,1,1165,True
91,1166,1,1166,True
91,1167,1,1167,True


In [67]:
%%sql
SELECT a.name artist,
       COUNT(DISTINCT(plt.playlist_id)) number_of_playlists
  FROM playlist_track plt
 INNER JOIN track t
    ON plt.track_id = t.track_id
 INNER JOIN album al
    ON t.album_id = al.album_id
 INNER JOIN artist a
    ON al.artist_id = a.artist_id
 GROUP BY artist
 ORDER BY number_of_playlists DESC
 LIMIT 5;
 


Done.


artist,number_of_playlists
Eugene Ormandy,7
Academy of St. Martin in the Fields & Sir Neville Marriner,6
Berliner Philharmoniker & Herbert Von Karajan,6
English Concert & Trevor Pinnock,6
The King's Singers,6


## Artists in Playlists

The artist in the most playlists is Eugene Ormandy with 7.

In [121]:
%%sql

SELECT CASE
           WHEN il.track_id IS NULL THEN "Not purchased"
           ELSE "Purchased"
           END AS was_purchased,
        COUNT(DISTINCT(t.track_id)) count,
        CAST(COUNT(DISTINCT(t.track_id)) as float) / (SELECT COUNT(*)
           FROM track) percentage
  FROM track t
  LEFT JOIN invoice_line il
    ON t.track_id = il.track_id
 GROUP BY was_purchased


Done.


was_purchased,count,percentage
Not purchased,1697,0.4844419069369112
Purchased,1806,0.5155580930630888


## Purchased Tracks

Roughly half of all tracks available have been purchased.

In [125]:
%%sql

SELECT mt.name,
       COUNT(*),
       ROUND(CAST(COUNT(*) AS float) / (SELECT COUNT(*)
                                    FROM track), 2) percentage
  FROM track t
 INNER JOIN media_type mt
    ON t.media_type_id = mt.media_type_id
 GROUP BY mt.name
 

Done.


name,COUNT(*),percentage
AAC audio file,11,0.0
MPEG audio file,3034,0.87
Protected AAC audio file,237,0.07
Protected MPEG-4 video file,214,0.06
Purchased AAC audio file,7,0.0


In [140]:
%%sql

SELECT COUNT(*) number_tracks_sold,
        CASE
          WHEN mt.name LIKE "%Protected%" THEN "Protected"
          ELSE "Unprotected"
          END AS protection,
       ROUND(CAST(COUNT(*) AS float) / (SELECT COUNT(*)
                                    FROM invoice_line), 2) percentage
  FROM invoice_line il
  LEFT JOIN track t
    ON il.track_id = t.track_id
  LEFT JOIN media_type mt
    ON t.media_type_id = mt.media_type_id
 GROUP BY protection

Done.


number_tracks_sold,protection,percentage
442,Protected,0.09
4315,Unprotected,0.91


## Protected VS Unprotected

Protected music files make up about 13% of all tracks available in the music store, less than 10% of track sales.  Protection is not likely to affect popularity.